#   Adversarial Search in Game Playing

Mathematical game theory, a branch of economics, views any multiagent environment as a game, provided that the impact of each agent on the other is 'significant' regardless of whether they're competitive or cooperative. The type of games we usually study in Artificial Intelligence are special kind of games which are referred to as __zero-sum games__. These type of games are deterministic, turn-taking, two-player, of __perfect information__. When we say sero-sum, we mean a game situation in which each participant's gain or loss of utility is exactly balanced by the losses or gains of the utility of the other participants. When we say perfect information, we mean that we have complete knowledge of all the states in the future, actions possible from each state, and the value achieved at the end of the game. For example, tic-tac-toe, Go, and chess are an example of a zero-sum game of perfect information. On the other hand, games like Poker are called __imperfect information__ games, because each player's cards are hidden from other players and therefore it's not possible to know which states are possible in the future. 


Before we start discussing the adversarial search techniques in game-playing, we need to define what does game playing mean. According to Peter Norvig's __Artificial Intelligence - A Modern Approach__, game-playing can formally be defined as a kind of search problem with the following elements:

> - S$_{0}$: The __initial state__, which defines how the game is set up at the start,
> - Player(s): Defines which players has the turn in a state,
> - Actions(s): Returns the set of legal moves at state s,
> - Result(s, a): The __transition model__, which defines the result of a move,
> - Terminal-Test(s): A __terminal test__, which is true when the game is over and false otherwise. States where the game has ended are called __terminal states__.
> - Utility(s, p): Returns the __value__ of the game for player p at the terminal state s.


Having defined what game-playing is, we need to understand the set-up on which (adversarial) search algorithms can work. We usually use a Tree data structure to formalize the game setting. The initial state, Actions function, and Result function define a game tree for a given game. In a game tree, each node corresponds to game states and edges corresponds to possible action from that state (node). For example, the tree search for tic-tac-toe would look like this: 

![title](images/tictactoe.png)

For tic-tac-toe, the number of terminal game states is upper-bounded by 9! = 362,880 terminal states, but for games like Chess and Go, the number of nodes in a game tree search is extremely high (higher than 10$^{40}$). Now let's move on to how we define optimal moves in a game.

### Optimal Move Strategies

Roughly speaking, an optimal game playing strategy leads to outcomes at least as good as any other strategy when playing against an infallible opponent. We will discuss a few major optimal strategies algorithms like MiniMax, Alpha-Beta Pruning, and the Monte-Carlo Tree Search. Since the main focus of this post is explain a major component of AlphaZero, we will not go in depth on MiniMax and Alpha-Beta pruning, but just provide enough information for the reader to have an idea of what these algorithms aim to achieve.

#### MiniMax Algorithm:

To understand MiniMax algorithm, let's first define what a __minimax value__ and a __minimax decision__ is. Since we're only considering two-players games like tic-tac-toe, chess, Go etc, let's call one player MAX and the other MIN. Take a look at the game search tree below. 

![title](images/minmax.png)

Here, assuming MAX takes a turn at the initial state S$_{0}$ (root node), it would select the move that leads to its child state with the highest _value_ (state B from action a$_{1}$). Now MIN will take a turn at the next level (depth = 1), and it would select the move that leads to its child state with the lowest _value_ (state from action b$_{1}$) and so on. Therefore, we define the __minimax value__ as the max value state available for MAX and min value state available for MIN. But how do we decide these values? These values could be as simple as the sum of wins from a given state. For example, in tic tac toe, the terminal nodes have values -1 (min wins), 0 (draw), or +1 (max wins). So the values would be -1, 0, or +1. Formally, __minimax value__ is defined as follows:

![title](images/minmaxvalue.png)

And __minimax decision__ is defined as a decision (move) based on __minimax value__. 

The MiniMax Algorithm computes the minimax decision from the current state. It uses a simple recursive computation of the minimax values of each successor state, directly implementing the defining equations. The recursion proceeds all the way down to the leaves of the tree, and then the minimax values are backed up through the tree as the recursion
unwinds. The minimax algorithm performs a complete depth-first search of the game tree. If the maximum depth of the game tree is m, and there are b legal moves at each point, then __the time complexity of the minimax algorithm is O(b$^{m}$)__ and __the space complexity is O(bm)__. Though minimax is provable an optimal game strategy, we can see how it becomes computatationally unfeasible to carry out on game trees with high depth and high branching factor. This is why we would need improvements on minimax algorithm, which we'll discuss later in this post. Here's the pseudo code and a basic implementation of the minimax algorithm. 

![title](images/minmaxalg.png)

In [1]:
class MiniMax:
    def __init__(self, game_tree):
        self.game_tree = game_tree
        self.root = game_tree.root
        self.currentNode = None 
        self.successors = []

    def minimax_value(self, node):
        """Takes as input the current state and returns the minimax decision"""
        best_val = self.max_value(node) 

        successors = self.getSuccessors(node)
        print("Running MiniMax at node: " + str(best_val))
        best_move = None
        for move in successors:
            if move.value == best_val:
                best_move = move
                break
        return best_move


    def max_value(self, node):
        """Takes as input the current state and returns the state with the max value"""
        
        print("Checking for MAX at node: " + node.Name)
        if self.isTerminal(node):
            return self.getUtility(node)
        
        max_value = -(float('inf')) #set the initial max_value to negative infinity (update when larger value found)
        successors_states = self.getSuccessors(node)
        for state in successors_states:
            max_value = max(max_value, self.min_value(state))
        return max_value

    def min_value(self, node):
        """Takes as input the current state and returns the state with the min value"""
        print("Checking for MIN at node: " + node.Name)
        if self.isTerminal(node):
            return self.getUtility(node)

        min_value = float('inf') #set the initial min_value to positive infinity (update when larger value found)
        successor_states = self.getSuccessors(node)
        for state in successor_states:
            min_value = min(min_value, self.max_value(state))
        return min_value

    
    """Helper Functions"""
    
    def getSuccessors(self, node):
        """Takes as input the current state and returns the children states"""
        assert node is not None
        return node.children

    def isTerminal(self, node):
        """Takes as input the current state and returns True if it's a terminal state"""
        assert node is not None
        return len(node.children) == 0

    def getUtility(self, node):
        """Takes as input the current state and returns its value"""
        assert node is not None
        return node.value

#### Alpha-Beta Pruning:

The problem with minimax strategy is that the number of game states it examines for evaluation grows exponentially in terms of the depth of the tree. So even for games like tic-tac-toe, the upper bound for the number of state evaluations would be O(9$^{9}$) because the maximum number of moves are 9 (at root S$_{0}$) and maximum depth is also 9. Unfortunately, we can't do better than the exponent since we're essentially performing a depth-first search. However, we can cut it in half. The key is that it is possible to compute the minimax strategy without looking at every state. The technique to achieve so is called __Alpha Beta Pruning__. Alpha and Beta are the two variable that indicate at each state the best (in term of both max and min) we can do and these help us 'prune' the game tree search that we know is not going to affect the minimax decision. 

More formally,
- $\alpha$ = the value of the best (highest value) choice we have found so far at any point along the path for MAX
- $\beta$ = the value of the best (lowest value) choice we have found so far at any point along the path for MIN

The general strategy is as follows: consider a node n somewhere in the tree, such that player has choice of moving to that node. If the player has a better choice m either at the parent node of n or at any choice point further up, then n will never be reached in actual play. So once we have found enough about n by evaluating some if its descendants, we can prune it. Alpha-Beta Pruning updates $\alpha$ and $\beta$ as it goes along and prunes the remaining branches at a node as soon as the value of the current node is known to be worse than the current $\alpha$ or $\beta$ for MAX or MIN respectively. 

Following is the pseudocode and a basic implementation of Alpha-Beta Pruning. 

![title](images/alphabeta.png)

In [2]:
class AlphaBeta:
    def __init__(self, game_tree):
        self.game_tree = game_tree
        self.root = game_tree.root

    def alpha_beta_search(self, node):
        best_val = -(float('inf'))
        beta = -(float('inf'))

        successors = self.getSuccessors(node)
        best_state = None
        for state in successors:
            value = self.min_value(state, best_val, beta)
            if value > best_val:
                best_val = value
                best_state = state
        print("AlphaBeta: Utility at Root Node: " + str(best_val))
        return best_state

    def max_value(self, node, alpha, beta):
        print("Check for Max at node: " + node.Name)
        if self.isTerminal(node):
            return self.getUtility(node)
        value = -(float('inf'))

        successors = self.getSuccessors(node)
        for state in successors:
            value = max(value, self.min_value(state, alpha, beta))
            if value >= beta:
                return value
            alpha = max(alpha, value)
        return value

    def min_value(self, node, alpha, beta):
        print("Check for Min at node: " + node.Name)
        if self.isTerminal(node):
            return self.getUtility(node)
        value = float('inf')

        successors = self.getSuccessors(node)
        for state in successors:
            value = min(value, self.max_value(state, alpha, beta))
            if value <= alpha:
                return value
            beta = min(beta, value)

        return value

    """Helper Functions"""

    def getSuccessors(self, node):
        assert node is not None
        return node.children

    def isTerminal(self, node):
        assert node is not None
        return len(node.children) == 0

    def getUtility(self, node):
        assert node is not None
        return node.value

Alpha Beta Pruning is definitely a poweful algorithm that achieves optimal game playing strategy. IBM's Deep Blue that defeated a chess world champion in 1996 was designed using Alpha-Beta search. For games like Go, the game tree complexity is immensely large (10$^{170}$ legal moves on a 19x19 board). Therefore, it becomes computationally infeasible to run Alpha-Beta search. Just to have an abstract idea of how the game search tree grows for Go, take a look at the image below. 

![title](images/gotree.gif)

Due to the high branching factor in the game tree for Go, Alpha-Beta Pruning does not stand as an effective choice. As a way around the high computattion cost attached with games like Go, we can do two things:

> - Use a heuristic function, which is a function that gives an estimate of the value of a state of the game. Instead of considering all the moves in a game, you can just consider moves out to some finite distance ahead, and then use the value of the heuristic to judge the value of the states you reached.

> - Use Rollouts. At a given state, just randomly simulate the game by taking turns for both the players until you reach a terminal state. The belief is that as the number of rollouts increase, the sample average win rate (no. wins / no. rollouts) will converge to the expected win rate from that state.

Since, a lot of complex games like Go lack any definitive heuristic are based more on human intuition, it is hard to come up with a single heuristic function that would evaluate the game states for us. This leads us to focus on the second option of using Rollouts. In particular, we will discuss another extremely powerful game tree search strategy - __the Monte Carlo Tree Search (MCTS)__. MCTS is the algorithm of choice for AlphaZero and hence we will dive deeper into this algorithm to understand why it is a better choice. 